# 02806 Final project 
> An analysis and visualization for novel COVID-19 virus

- toc: true 
- badges: true
- author: Georgios Zefkilis & Yucheng Ren 
- comments: false
- categories: [data_analysis, visualization]

> Tip: This page is generated from a Jupyter notebook, some of the code are hid under the hood, some of them can be shown by clicking the button `Show Code`. If you want to visit the complete notebook, please click the `view on github` button above.

# Introduction
 **should the scope would be to show the impact in global economy or how has affected Denmark in comparison with major countries** **how we define major?**

Undoubtedly the recent appearrance and expansion of COVID-19 virus has affected the lives of billions of people worldwide is many aspects. Goverments have been under constant challenge to reduce social interaction in order to mitigate the possibilities of virus transmission. Therefore, they have introduced hard measurements to face this severe situation which have significant impact to every body's live. 
    
Economy was the first area that affected from those measurements. The work culture had to change to meet the derivative of the goverments, which led companies to move faster towards digitilisation. As a result companies that weren't eager in such changes to face important financial issues forcing them in many cases to reduce their human resources. For other companies such travelling agencies or copmanies in hospitalitty sector, the hit was even harder since they rely their profits entirely on the people's need for entertainment, social exploration etc.. Therefore, they have completely or partially shut down their operation leading many people in unemployment.

The above constitutes common observations and may look discouranging and demotivating facts for many people. However, we can not conclude how big this impact is in each country's overall economy without an in depth investigation of actual facts. 
    
Upon that, we came to the desicion to analyse data from microeconomic and macroeconomic point of view in order to get a more clear understanding of how the virus has affected our economy.
    

To sum up, from this study we aim to provide a clear conclusion about the economic consequences due to COVID-19 which will be based on analysis of reliable sources. Through interactive and annotated graphs we want to give to the intendent  audience all the information needed in order to understand the impact of COVID-19 in economy in a simple and concine manner.  
    
    


In [2]:
# hide
# this block contains all the import packages
import pandas as pd
import numpy as np
import altair as alt
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from vega_datasets import data
import plotly.graph_objects as go
import matplotlib.colors as mcolors

In [41]:
# hide
# import data
path = 'data/'
pd.set_option('mode.chained_assignment', None)

imfGDP = pd.read_csv(path + 'imf-dm-export-20200423.csv')
stockOMX20 = pd.read_csv(path + 'OMX20.csv')
stockOMX25 = pd.read_csv(path + 'OMX25.csv')
stockCopenhagenAllShare = pd.read_csv(path + 'OMXCopenhagenAllshares.csv')
# data = pd.read_csv(path + 'MEI_CLI_24042020100529269.csv')
denEmploy = pd.read_csv(path + 'DenmarkEmploymentQuarterly.csv')
denUnemployMonthly = pd.read_csv(path + 'DenmarkUnemploymentMonthly.csv')
imfGlobalGdp = pd.read_csv(path + 'imf-global-gdp.csv')
unemployRate = pd.read_csv(path + 'unemploymentRate.csv')
full_clean_data = pd.read_csv('https://raw.githubusercontent.com/datasets/covid-19/master/data/countries-aggregated.csv')
world_data = pd.read_csv('https://raw.githubusercontent.com/datasets/covid-19/master/data/time-series-19-covid-combined.csv')
refrence = pd.read_csv('https://raw.githubusercontent.com/datasets/covid-19/master/data/reference.csv')
majorCountry = pd.read_csv(path + 'majorCountry.csv')

# Data Analysis

    
In the study we will analyse data from all the countries directly affected from COVID-19 giving more focus though in Denmark. 
We will start the study by presenting a statistical analysis of how the situation with regards to COVID-19 looks like in the most major countries. Then we will include financial data to explore whether there is a significant impact of the virus in our economy and which countries specifically have affected the most.
In order to carry out the analysis we will use data from IMF, OECD and other sources which can be found at the end of the page.
The reason we chose those datasets was that we believe they contain all the information needed to obtain the required outcome about the fincanial impact of COVID-19.



## COVID-19 analysis

In this section, we will dive more into COVID-19 data to present the current situation of virus by illustrating the the numbers of confirmed and death cases across major countries. Then with help of interactive represenation of those numbers we will try to understand the spread rate and distribution of COVID-19.

### Data preparation

In the following table is shown a sample of the data regarding COVID-19. The dataset contains columns with the countries, confirmed and recovered cases as well as overall deaths per country.


In [4]:
# hide_input
full_clean_data.tail()

Date             Country  Confirmed  Recovered  Deaths
18310  2020-04-29  West Bank and Gaza        344         71       2
18311  2020-04-29      Western Sahara          6          5       0
18312  2020-04-29               Yemen          6          1       0
18313  2020-04-29              Zambia         97         54       3
18314  2020-04-29            Zimbabwe         32          5       4

In order to extract more information as possible from the dataset it is necessary to combine several datasets.
By doing so, we include columns referring to daily new cases, new deaths and new recovered cases. Other, than that an investigation for missing values and treatment of those it is also a requirement to bring the dataset in form ready for analysis. In the present study the missing values were filled with zeros. It considered the best way to treat such a values because if for example the missing values were filled with the mean, mode or median could lead to false interpration of the results.  **maybe present some stats first**


In the following table it is shown the final dataset about COVID-19 looks after the preprossesing. 

In [5]:
# collapse-show
# data processing to create Active, New cases, New deaths, New recovered
full_clean_data['Active'] = full_clean_data['Confirmed'] - full_clean_data['Recovered'] - full_clean_data['Deaths']

countries = ['US', 'Italy', 'China', 'Spain', 'France', 'Iran', 'United Kingdom', 'Denmark']
selected_data = full_clean_data[full_clean_data['Country'].isin(countries)]

for i in selected_data.index:
    date = selected_data.loc[i, 'Date']
    country = selected_data.loc[i, 'Country']
    date = datetime.strptime(date, '%Y-%m-%d')
    yesterday = datetime.strftime(date - timedelta(1), '%Y-%m-%d')
    yesterdayData = selected_data.loc[(selected_data.Date == yesterday) & (selected_data.Country == country)]
    if len(yesterdayData) <= 0:
        selected_data.loc[i, 'New cases'] = 0
        selected_data.loc[i, 'New deaths'] = 0
        selected_data.loc[i, 'New recovered'] = 0
        continue
    yesterdayData = yesterdayData.iloc[0]
    selected_data.loc[i, 'New cases'] = selected_data.loc[i, 'Confirmed'] - yesterdayData.Confirmed
    selected_data.loc[i, 'New deaths'] = selected_data.loc[i, 'Deaths'] - yesterdayData.Deaths
    selected_data.loc[i, 'New recovered'] = selected_data.loc[i, 'Recovered'] - yesterdayData.Recovered

selected_data = selected_data.fillna(value=0)
selected_data['New cases'] = selected_data['New cases'].astype(int)
selected_data['New deaths'] = selected_data['New deaths'].astype(int)
selected_data['New recovered'] = selected_data['New recovered'].astype(int)

In [6]:
# hide_input
selected_data.tail()

Date         Country  Confirmed  Recovered  Deaths  Active  \
18210  2020-04-29            Iran      93657      73791    5957   13909   
18214  2020-04-29           Italy     203591      71252   27682  104657   
18286  2020-04-29           Spain     236899     132929   24275   79695   
18301  2020-04-29              US    1039909     120720   60967  858222   
18305  2020-04-29  United Kingdom     166441        857   26166  139418   

       New cases  New deaths  New recovered  
18210       1073          80           1352  
18214       2086         323           2311  
18286       4771         453           9026  
18301      27327        2612           4784  
18305       4091        4421             44

### Statistical analysis



**include basic statistic tables about the data**

### COVID-19 presentation in Denmark

The figure below illustrates the total confirmed cases and deaths in Denmark from the day the virus appeared in the country (approximately January 22) until April 23. **we can either update the date or just stop to the day the lockdown stopped**

It seems that even during the lock down (10/03/2020 - 20/03/20) the number of confirmed cases and deaths showed an increasing trend. Although it has to be highlighted that the purpose of lock down was to keep these numbers as low as possible in order not to exceed the capacity of the cases that the health system can handle. **maybe we can find data about that and include them to see if they get the target**

In [7]:
# collapse-hide
denmark = selected_data[selected_data['Country'] == 'Denmark']

base = alt.Chart(denmark).mark_bar().encode(
    x='monthdate(Date):O',
).properties(
    width=500
)

red = alt.value('#f54242')
base.encode(y='Confirmed').properties(title='Total Confirmed') | base.encode(y='Deaths', color=red).properties(title='Total Deaths')

alt.HConcatChart(...)

Moving forward to the figures below, it is observed the number of new incidents(left pane) and deaths (right pane) in Denmark during the same period. 

When the lock down implemented (*around 10th of March*) and until the *13th* of the same month *170* new cases were recorded daily. Whereas on the *14th of March* a significant drop of approximatelly *75%* of the cases recorded is observed. Between, *March the 24th and April the 9th* the number of confirmed cases reached its peak with an average of *300* cases per day. Then the recorded cases began to drop again until today where they have reached of an average *150* cases per day(*10/05/2020 - 23/05/2020*).

The number of daily deaths reached its highest numbers between *3rd and 9th of April* and dropped by approximatelly *50%* after that. By today the number of deaths per day doesn't exceed the 9 deaths. Overall, we can see that the measurments against the virus yielded in reduction of deaths and confirmed cases after its implementation. 

In [8]:
# collapse-hide
base.encode(y='New cases').properties(title='Daily New Cases') | base.encode(y='New deaths', color=red).properties(title='Daily New Deaths')

alt.HConcatChart(...)

**Death rate** calculation formula: Death No. / Confirmed No.

In [56]:
# collapse-hide
# data processing
selected_data['DeathRate'] = selected_data['Deaths'] / selected_data['Confirmed'] * 100
selected_data = selected_data.fillna(value=0)

# plot
highlight = alt.selection(type='single', on='mouseover',
                          fields=['Country'], nearest=True)

base = alt.Chart(selected_data).transform_filter(
    alt.datum.Country != 'Iran'
).encode(
    x='Date:T',
    y=alt.Y('DeathRate:Q', title= 'Death Rate %'),
    color='Country:N',
    tooltip = [alt.Tooltip('DeathRate'),
               alt.Tooltip('Country'),
              ],
)

points = base.mark_circle().encode(
    opacity=alt.value(0)
).add_selection(
    highlight
).properties(
    width=600
)

lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1), alt.value(3))
)

points + lines

alt.LayerChart(...)

In [10]:
# collapse-hide
# data processing
start_date = datetime.strptime('2020-01-22', '%Y-%m-%d')
for index, row in selected_data.iterrows():
    date = datetime.strptime(row['Date'], '%Y-%m-%d')
    selected_data.loc[index, 'Day'] = (date - start_date).days
    
selected_data['Day'] = selected_data['Day'].astype(int)
# plot
select_date = alt.selection_single(
    name='select', fields=['Day'], init={'Day': 0},
    bind=alt.binding_range(min=0, max=selected_data.Day.max(), step=5)
)
alt.Chart(selected_data).mark_point(filled=True).encode(
    alt.X('Confirmed', scale=alt.Scale(zero=False)),
    alt.Y('Deaths', scale=alt.Scale(zero=False)),
    alt.Size('Active'),
    alt.Color('Country'),
    alt.Order('Confirmed', sort='descending'),
    tooltip = [alt.Tooltip('Confirmed'),
               alt.Tooltip('Deaths'),
               alt.Tooltip('Active')
              ],
).properties(
    width=400,
    height=300
).add_selection(select_date).transform_filter(select_date)

alt.Chart(...)

### Overlook of COVID-19 current distribution worldwide

In [59]:
# collapse-hide
# data preperation, combine refrence dataset to virus dataset to obtain area code for map plot
refrence = refrence.rename(columns={'Country_Region': 'Country/Region'})
most_recent_data = world_data[world_data['Date'] == world_data['Date'].max()]
most_recent_data = most_recent_data[['Date', 'Country/Region', 'Confirmed','Recovered','Deaths']]
grouped = most_recent_data.groupby('Country/Region').sum()

result = grouped.join(refrence.set_index('Combined_Key'), on='Country/Region')
result = result.fillna(value=0)
result['code3'] = result['code3'].astype(int)

# plot
alt.Chart(alt.topo_feature(data.world_110m.url, 'countries')).mark_geoshape(
    stroke='#aaa', strokeWidth=0.25
).transform_lookup(
    lookup='id', from_=alt.LookupData(data=result, key='code3', fields=['Confirmed'])
).encode(
    alt.Color('Confirmed:Q',
              scale=alt.Scale(domain=[0, result.Confirmed.max()/10], clamp=True), 
              legend=alt.Legend(format='')),
    alt.Tooltip('Confirmed:Q')
).project(
    type='equirectangular'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

### COVID-19 Spreading Among Major Countries


Below it is illustrated how the COVID-19 has been evolved among major countries and how they compared to Denmark. 
The term major reflects to the number of cases recorded in those countries. Those countries namely are China (where the COVID-19 crisis started) USA, France, Iran, United Kingdom, Italy and Spain. Even though the presentation of those countries takes only the number of cases into account, it is interesting that apart from Spain, Iran and Denmark the rest constitute part of the  group of seven largest IMF-advanced economies of the world.

In China where the COVID-19 first appeared, shows a high increase in number of cases per day during February and  in relatively short period of time archives to diminish those numbers. On

**The graphs look a bit misleading, I would suggest to replace them**


In [12]:
# collapse-hide
# plot
interval = alt.selection_interval()

circle = alt.Chart(selected_data).mark_circle().encode(
    x='monthdate(Date):O',
    y='Country',
    color=alt.condition(interval, 'Country', alt.value('lightgray')),
    size=alt.Size('New cases:Q',
        scale=alt.Scale(range=[0, 3000]),
        legend=alt.Legend(title='Daily new cases')
    ) 
).properties(
    width=1000,
    height=400,
    selection=interval
)

bars = alt.Chart(selected_data).mark_bar().encode(
    y='Country',
    color='Country',
    x='sum(New cases):Q'
).properties(
    width=1000
).transform_filter(
    interval
)

circle & bars

alt.VConcatChart(...)

## Macroeconomic

**should we show only for Denmark or globally**

In this section we will attempt to perform an economic analysis from a macroeconimic point of view and in relation to the COVID-19 analysis above, we will try to come up with the potential coclusions on how the spread of the virus has affected the global economy. A closer look to Denmark will be given in this section as well. **take a look on that again**.

Macroeconomics is a branch of economics that studies how an overall economy behaves (focuses on the large scale). More presicely, macroeconomics studies economy-wide phenomena such as inflation, price levels, rate of economic growth, national income, gross domestic product (GDP), and changes in unemployment 
([Investopedia](https://www.investopedia.com/terms/m/macroeconomics.asp)).


### Stock Market
**i believe this needs more discussion and what really want to show with the stocks**

**also i couldn't find how stocks related to macroeconomy**

Talk about the stock market



In [13]:
# hide
# stock data preprocessing
stockOMX20['Symbol'] = 'OMX 20'
stockOMX25['Symbol'] = 'OMX 25'
stockCopenhagenAllShare['Symbol'] = 'Copenhagen All Shares'
stockAll = pd.concat([stockOMX20, stockOMX25, stockCopenhagenAllShare])
stockAll['Date'] = pd.to_datetime(stockAll.Date)
stockAll = stockAll.sort_values(by=['Symbol', 'Date'])
stockAll['Price'] = stockAll['Price'].str.replace(',', '')
stockAll['Price'] = stockAll['Price'].astype(float)

In [62]:
# collapse-hide
line = alt.Chart(stockAll).mark_line(interpolate='basis').encode(
    x='Date',
    y='Price',
    color='Symbol',
)

nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['Date'], empty='none')

selectors = alt.Chart(stockAll).mark_point().encode(
    x='Date',
    opacity=alt.value(0)
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'Price', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(stockAll).mark_rule(color='gray').encode(
    x='Date',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
alt.layer(
    line, selectors, points, rules, text
).properties(
    width=600, height=300
)

alt.LayerChart(...)

## GDP Inflation & unemployment data 

Major countrys' GDP Inflation and unemployment annual change rate data from IMF includes forecast of 2020 and 2021

**it would be good to illustrate how it changes along with the unemployment rate**

In [106]:
# collapse-hide
# data preprocessing
def extract_data(df, subject):
    dates = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']
    d = {'Date': dates, 'Value': [df[date] for date in dates]}
    values = []
    countries = []
    _dates = []
    for country in df.Country.unique():
        tmp = df.loc[df.Country == country]
        for date in dates:
            countries.append(country)
            _dates.append(date)
            values.append(float(tmp[date]))
    
    rv = pd.DataFrame.from_dict({'Date': _dates, 'Country': countries, 'Value': values})
    rv['subject'] = subject
    return rv

unemploy = majorCountry.loc[majorCountry['Subject Descriptor'] == 'Unemployment rate']
unemploy = extract_data(unemploy, 'unemployment')
inflation = majorCountry.loc[majorCountry['Subject Descriptor'] == 'Inflation, average consumer prices']
inflation = extract_data(inflation, 'inflation')
gdp = majorCountry.loc[majorCountry['Subject Descriptor'] == 'Gross domestic product, constant prices']
gdp = extract_data(gdp, 'gdp')

# A dropdown filter
countries = list(majorCountry.Country.unique())
country_dropdown = alt.binding_select(options=countries)
country_select = alt.selection_single(fields=['Country'], bind=country_dropdown, name="Select")

filter_gdp = alt.Chart(gdp, width=300, height=300, title='GDP Growth of Major Countries').mark_line(point=True).encode(
    alt.X('Date:T'),
    alt.Y('Value:Q', title= 'Growth Rate %'),
    color='Country',
    tooltip = [alt.Tooltip('Value:Q')]
).add_selection(country_select).transform_filter(country_select)

# umemployment plot
filter_unemployment = alt.Chart(unemploy, width=300, height=300, title='Unemployment Change of Major Countries').mark_line(point=True).encode(
    alt.X('Date:T'),
    alt.Y('Value:Q', title= 'Growth Rate %'),
    color='Country',
    tooltip = [alt.Tooltip('Value:Q')]
).add_selection(country_select).transform_filter(country_select)

# inflation plot
filter_inflation = alt.Chart(inflation, width=300, height=300, title='Inflation Change of Major Countries').mark_line(point=True).encode(
    alt.X('Date:T'),
    alt.Y('Value:Q', title= 'Growth Rate %'),
    color='Country',
    tooltip = [alt.Tooltip('Value:Q')]
).add_selection(country_select).transform_filter(country_select)


(filter_gdp | filter_unemployment | filter_inflation)

alt.HConcatChart(...)

# data source 

https://ilostat.ilo.org

https://www.investing.com

https://www.imf.org

http://www.oecd.org

https://github.com/datasets/covid-19



# References

1. [Investopedia](https://www.investopedia.com/terms/m/macroeconomics.asp)